In [1]:
from skimage.feature import hog
from skimage import io,color
from skimage.transform import resize
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn import metrics
import os
from  LBP_descriptor import LocalBinaryPatterns
import commonfunctions as cf
import cv2
import pandas as pd
import csv
from skimage.feature import greycomatrix, greycoprops

# Get our training data 
X_train: features of training data.\
Y_train: labels of training data (1-->F, 0--> M).


In [2]:

# ICDAR LABELS 
labels_ICDAR=[]
with open("our dataset/train_answers.csv", 'r') as file:
    csvreader = csv.reader(file, delimiter=',')
    rows= np.array(list(csvreader))[1:].astype(float).astype(int)
for row in rows:
    labels_ICDAR.append(row[1])
    
def get_label_ICDAR(img):
    if img[0]=='0':
        if img[1]=='0': 
            return labels_ICDAR[int(img[2])-1]
        else: 
            return labels_ICDAR[int(img[1:3])-1]
    else: 
         return labels_ICDAR[int(img[0:3])-1]
        


def read_labels(path): 
    y=[]
    files = os.listdir(path)

    for file in files:
        if file[0]=='F':
            y.append(0)
        elif file[0]=='M':
            y.append(1)
        else: 
            y.append(get_label_ICDAR(file[1:4]))
          
    y=np.array(y).astype(float)
    return y 

Y_train= read_labels("Training_data/")
Y_test= read_labels("Test_data/")

# HOG feature

In [3]:
def HOG(img):
    img = np.array(resize(img,(128,64))) 
    feature_vector, hog_image = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(3,3), visualize=True)
    return feature_vector,hog_image


In [4]:
# TRY GLCM ONLY
X_train=[]
 
files = os.listdir("Training_data/")
for file in files:
    # read the image
    img = cv2.imread("Training_data/"+file )
    # ------------------- HOG feature------------------------
    feature_vector,hog_image=HOG(img)
    X_train.append(feature_vector)
#convert to numpy array
X_train=np.array(X_train)
   

In [5]:
#TEST GLCM
X_test=[]
files = os.listdir("Test_data/")

for file in files:
   
    # read te img
    img = io.imread("Test_data/" +file )
    # ------------------- HOG feature------------------------
    feature_vector,hog_image=HOG(img)
    X_test.append(feature_vector)
    
X_test=np.array(X_test)

# Classification:

# 1. Random forest classifier

In [11]:

clf=RandomForestClassifier(n_estimators=2000)
clf.fit(X_train,Y_train)
Y_Predicted=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_Predicted)*100,"%")


Accuracy: 60.69364161849711 %


# 2. Linear SVM classifier

In [14]:
clf=LinearSVC(C=300.0, random_state=42)
clf.fit(X_train,Y_train )
Y_Predicted=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_Predicted)*100,"%")

Accuracy: 61.849710982658955 %


c:\Users\dell\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


# 3. Adaboost classifier

In [8]:
clf=AdaBoostClassifier(n_estimators=3000)
clf.fit(X_train,Y_train)
Y_Predicted=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_Predicted)*100,"%")

Accuracy: 53.179190751445084 %


# 4. KNN classifier

In [9]:
accuracies=[]
for k in range(1,30):
    clf=KNeighborsClassifier(n_neighbors = k)
    clf.fit(X_train,Y_train)
    Y_Predicted=clf.predict(X_test)
    accuracies.append(metrics.accuracy_score(Y_test, Y_Predicted)*100)

print("Accuracy:",accuracies[np.argmax(accuracies)],"%")

Accuracy: 58.38150289017341 %
